In [ ]:
import JupyterNotebooksLib as slicernb
import slicer
import os 
from DICOMLib import DICOMUtils
import numpy as np
import zipfile

from glob import glob
from natsort import natsorted
from tqdm import tqdm

# Set image viewer size to 50% screen size
# slicernb.AppWindow.setWindowSize(scale=0.5)
# slicernb.AppWindow.setContents("viewers")

# Clear scene
slicer.mrmlScene.Clear(False)

# Unzip and organize files 

In [ ]:
#input folder 
input_folder = "C:\\Users\\Sepehr\\3dsegmentationProjects\\nn-unet-organize-dataset-for-training\\Segmentation database"
intermediary_folder = "./temp_unzip"

if  not os.path.exists(intermediary_folder):
    os.mkdir(intermediary_folder)

In [ ]:
# for zip_file in os.listdir(input_folder):
#     if "(" in zip_file or ")" in zip_file:
#         continue;
#     zip_path = f"{input_folder}/{zip_file}"
#     Case_id = zip_file.split("_")[0]

#     if "_segmented_structures" in zip_file: 
#         out_path = f"{intermediary_folder}/{Case_id}_segmentations"
#     else:
#         out_path = f"{intermediary_folder}/{Case_id}_volumes"
        
#     with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#                     zip_ref.extractall(out_path)

# organize file names throw out unuasable cases 

In [ ]:
input_cases = [] 


volumes_paths = list(natsorted(glob(os.path.join(intermediary_folder,  "*"+ f"_volumes"))))

for vol in volumes_paths: 

    seg_path = vol.replace ("_volumes","_segmentations")

    if (os.path.exists(seg_path)):
        input_cases.append ({"vol":vol,"seg":seg_path})
    else:
        print ("cannot find matching segmentation case will be skipped")



In [ ]:
input_cases[0]["vol"].split("\\")[-1].split("_")[0]

# using slicer to convert to nii

### will need to alter segmentation nodes to reflect changes in classes you are trying to segment

In [ ]:
nii_path = "./nii_ds_1"
nii_path_volumes = f"{nii_path}/Volumes"
nii_path_segmentations = f"{nii_path}/Masks"

try:
    os.mkdir(nii_path)
    os.mkdir(nii_path_volumes)
    os.mkdir(nii_path_segmentations)
except:
    print ("these directories already exist")

In [ ]:
def get_model_class_from_file (file): 
    if "tooth_" in file: 
        #print (file.split("tooth_")[1].split("_")[0])
        return file.split("tooth_")[1].split("_")[0]

    else: 
        # CaseId = file.split("_") [-1].split(".)
        # #print (file.split(f"{CaseId}_")[-1].split("_")[0])
        return file.split("_") [-1].split(".")[0]

In [ ]:
get_model_class_from_file ("304846_canal.stl")
#"304846_canal.stl".split("_")

In [ ]:
def convert_to_nnunet_lable_structure (lable_dict,referenceVolumeNode):

    all_segment_np = [] 
    for lable, node in lable_dict.items():
        name = node.GetName()
    
        segmentationLblmap  = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLLabelMapVolumeNode")
        segmentationLblmap.SetName(name+"_lablemap")
        slicer.modules.segmentations.logic().ExportVisibleSegmentsToLabelmapNode(node, segmentationLblmap, referenceVolumeNode)
        mask = slicer.util.arrayFromVolume(segmentationLblmap)
        mask[mask > 0] = lable
    
        all_segment_np.append(mask.copy())



    final_out = all_segment_np[0] 
    for i in range (1,6):
        indices  = np.where (final_out <1)
        final_out [indices] = all_segment_np[i][indices]

    segmentationLblmap  = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLLabelMapVolumeNode")
    slicer.modules.segmentations.logic().ExportVisibleSegmentsToLabelmapNode(lable_dict[5], segmentationLblmap, referenceVolumeNode)
    
    slicer.util.updateVolumeFromArray(segmentationLblmap, final_out)

    return segmentationLblmap

    

In [ ]:
maxillary_teeth_lables = ["11","12","13","14","15","16","17","18","21","22","23","24","25","26","27","28"]
mandibular_teeth_lables = ["31","32","33","34","35","36","37","38","41","42","43","44","45","46","47","48"]

def convert_single_case_to_nii (dicom_series_path,segmentation_path):

    '''
    simply loading dicoms into a fresh scene 
    '''
    
    
    # # Clear scene
    slicer.mrmlScene.Clear(False)

    print(dicom_series_path)
    print(segmentation_path)
    
    ##load dicom 
    loadedNodeIDs = []
    with DICOMUtils.TemporaryDICOMDatabase() as db:
        DICOMUtils.importDicom(os.path.abspath(dicom_series_path), db)
        patientUIDs = db.patients()
    
        print (patientUIDs)
        for patientUID in patientUIDs:
            loadedNodeIDs.extend(DICOMUtils.loadPatientByUID(patientUID))



    '''
    loading the segmentations into appropriate substructuresz 
    
    '''
    
    
    ## creating segmentation nodes
    maxillaNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentationNode")
    maxillaNode.CreateDefaultDisplayNodes()
    maxillaNode.SetName ("maxillaNode")
    
    mandibleNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentationNode")
    mandibleNode.CreateDefaultDisplayNodes()
    mandibleNode.SetName ("mandibleNode")
    
    maxillaryTeethNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentationNode")
    maxillaryTeethNode.CreateDefaultDisplayNodes()
    maxillaryTeethNode.SetName ("maxillaryTeethNode")
    
    mandibularTeethNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentationNode")
    mandibularTeethNode.CreateDefaultDisplayNodes()
    mandibularTeethNode.SetName ("mandibularTeethNode")
    
    canalNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentationNode")
    canalNode.CreateDefaultDisplayNodes()
    canalNode.SetName ("canalNode")
    
    sinusNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentationNode")
    sinusNode.CreateDefaultDisplayNodes()
    sinusNode.SetName ("sinusNode")
    
    
    
    ## importing stls and binding them to the appropriate node 
    nodes = []
    node_names = []
    for seg in os.listdir(segmentation_path):
        if (".stl" in seg):
            mesh_path = f"{segmentation_path}/{seg}"
    
            class_name = get_model_class_from_file(seg)
            # print (seg)
    
            if class_name in maxillary_teeth_lables:
                slicer.modules.segmentations.logic().ImportModelToSegmentationNode(slicer.util.loadModel(mesh_path), maxillaryTeethNode)
            elif class_name in mandibular_teeth_lables:
                slicer.modules.segmentations.logic().ImportModelToSegmentationNode(slicer.util.loadModel(mesh_path), mandibularTeethNode)
            elif class_name == "maxilla":
                slicer.modules.segmentations.logic().ImportModelToSegmentationNode(slicer.util.loadModel(mesh_path), maxillaNode)
            elif class_name == "mandible":
                slicer.modules.segmentations.logic().ImportModelToSegmentationNode(slicer.util.loadModel(mesh_path), mandibleNode)
            elif class_name == "sinus":
                slicer.modules.segmentations.logic().ImportModelToSegmentationNode(slicer.util.loadModel(mesh_path), sinusNode)
            elif class_name == "canal":
                slicer.modules.segmentations.logic().ImportModelToSegmentationNode(slicer.util.loadModel(mesh_path), canalNode)


    referenceVolumeNode = slicer.mrmlScene.GetNodeByID("vtkMRMLScalarVolumeNode1")
    lable_dict = {1:sinusNode,2:canalNode , 3:mandibleNode,4:maxillaNode, 5:maxillaryTeethNode,6:mandibularTeethNode}

    segmentation_lable_map = convert_to_nnunet_lable_structure(lable_dict,referenceVolumeNode)

    caseID = dicom_series_path.split("\\")[-1].split("_")[0]
    volume_save_path = f"{nii_path_volumes}/{caseID}_volume.nii.gz"
    seg_save_path =  f"{nii_path_segmentations}/{caseID}_seg.nii.gz"

    slicer.util.exportNode(referenceVolumeNode, volume_save_path , {"useCompression": 0})
    slicer.util.exportNode(segmentation_lable_map, seg_save_path , {"useCompression": 0})

    print ("one case sucessfully converted")
     


        

In [ ]:
def batch_nii_convert (in_case_files):

    for case in tqdm(in_case_files):
        try:
            convert_single_case_to_nii (case["vol"],case["seg"])
        except:
            print (f"somthing is wrong with case {case} probably a missing segmentation")
    print ("done converting all cases to nii")
    

In [ ]:
batch_nii_convert (input_cases)

In [ ]:
# convert_single_case_to_nii (input_cases[0]["vol"],input_cases[0]["seg"])

# building nnunet_dataset from nii

In [ ]:
#making dataset Folder 

dataset_id = 125
dataset_name = "Max-Mand-Sinus-Teeth" 
Dataset_folder_name = f"Dataset{dataset_id}_{dataset_name}"

if (not os.path.exists(f"./nnUNet_raw/{Dataset_folder_name}")):
    os.mkdir(f"./nnUNet_raw/{Dataset_folder_name}")

train_input_dir = f"./nnUNet_raw/{Dataset_folder_name}/imagesTr"
train_lable_dir = f"./nnUNet_raw/{Dataset_folder_name}/labelsTr"

if not os.path.exists(train_input_dir):
    os.mkdir(train_input_dir)
if not os.path.exists(train_lable_dir):
    os.mkdir(train_lable_dir)




In [ ]:
import shutil
volume_list = os.listdir(nii_path_volumes)
seg_list = os.listdir(nii_path_segmentations)
# print (volume_list)

for volume in volume_list:
    volume_old_path = f"{nii_path_volumes}/{volume}"
    volume_nnunet_name = volume.replace ("_volume.nii.gz","_0000.nii.gz")
    volume_new_path = f"{train_input_dir}/{volume_nnunet_name}"

    #print (volume_nnunet_name)
    shutil.copy(volume_old_path, volume_new_path)


for segmentation in seg_list:
    segmentation_old_path = f"{nii_path_segmentations}/{segmentation}"
    segmentation_nnunet_name = segmentation.replace ("_seg.nii.gz",".nii.gz")
    segmentation_new_path = f"{train_lable_dir}/{segmentation_nnunet_name}"

    #print (segmentation_nnunet_name)
    shutil.copy(segmentation_old_path, segmentation_new_path)


In [ ]:
## creating Dataset Json 
import json 

dataset_metadata = {
 "channel_names": {  # formerly modalities
   "0": "CT", 
 }, 
 "labels": {  # THIS IS DIFFERENT NOW!
   "background": 0,
   "sinus": 1,
   "canal": 2,
   "mandible": 3,
   "maxilla":4,
   "maxillary_teeth":5,
   "mandibular_teeth":6  
 }, 
 "numTraining": len(volume_list), 
 "file_ending": ".nii.gz",
 "overwrite_image_reader_writer": "SimpleITKIO"  # optional! If not provided nnU-Net will automatically determine the ReaderWriter
 
}

json_data = json.dumps(dataset_metadata)

with open(f"./nnUNet_raw/{Dataset_folder_name}/dataset.json", "w") as outfile:
    outfile.write(json_data)